# Supervised Graph Learning

**Supervised learning** represents the majority of practical **machine learning** tasks. Thanks to more active and effective data collection, it is imore common to deal with labeled datasets. This also applies to graphs, where labels can be assigned to nodes, communities and structures so that we learn some mapping function between the input and label.

Code available at: https://github.com/PacktPublishing/Graph-Machine-Learning/tree/main/Chapter04

In SL, a training set has a sequence of ordered pairs *(x, y)* where x is a set of input features and y is the output label assigned to it, we then want to learn the mapping function of each *x* value to each *y* value. In some situations we have a smaller dataset of labeled instances and a larger set of unlabeled instances. Here, **semi-SL (SSL)** is proposed, where algorithmns learn dependencies of available labels to learn prediting functions for unlabeled samples. There are various algorithm types:
-  feature-based methods
-  shallow embedding methods
-  regularisation methods
-  graph neural networks

![4_1](./figures/4_1.jpg)

## Feature-based methods

Simple and powerful method for ML on graphs: consider encoding function as a simple embedding lookup. One simple way to do this is to exploid graph properties, and we know that graphs can be described by (exploiting) structural properties so important information "encoding" from the graph itself. A shallow approach acts in two steps:
1.  Select a set of *good* descriptive graph properties (e.g. avg. degree length, global efficiency etc..)
2.  Use such properties as input or a traditional ML algorithm

Unfortunately, there is no general definition of *good* descriptive properties, and their choice strictly depends on the specific problem to solve. 

Steps: 
1.  Convert StellarGraph to numpy adj matrices (networkx) and convert labels from Pandas series to numpy array.
2.  Compute global metrics to describe each graph, e.g. num edges, avg. cluster coefficient, global efficiency (can compute graph metrics with networkx)
3.  Exploit sckkit-learn to create train and test sets
4.  Train a ML alg, choose support vector machine (SVM), trained to minimise the difference between the predicted labels and the actual labels

For StellarGraph PROTEINS dataset, achieve about 80% F1-score, quite good for naive task.

## Shallow embedding methods

Subset of graph embedding methods that learn node, edge orgraph representation for a finite set of input data. Cannot be applied to other instances different from ones used to train the model. 

The main difference between unsupervised and supervised embedding models is the task they attempt to solve. If unsupervised shallow embedding algorithms try to learn a good graph/node/edge representation to build well defined clusters, supervised algorithms try to find the best solution for a prediction task, such as graph/node/edge classification.

We will see more supervised shallow embedding algorithms here.

## Label Propagation Algorithm
Used to solve node classification task, the algorithm propagates the label of a given node to its neighbours or to nodes having high probability of being reached from that node.

The only nonzero elements of the degree matrix are diagonal elements whose values represent the degree of the node represented by the row. Introduce transition matrix $L=$$D$<sup>$-1$</sup>=$A$ where l<sub>$ij$</sub>$\in L$ is the probability of reaching node $v_j$ from $v_i$. Probability of reaching an end node given a start node. 

Can see the probability of nodes being assigned labels. So we can perform *n* iterations, at each iteration *t*, the algorithm will compute the solution for that iteration:
$Y^t = LY$<sup>$t-1$</sup>
And stops when a certain condition is met.

However we can see the issues:
-  Possible to assigned only to nodes a probability associated with a label
-  The initial labels of values are different from the the one defined in $Y^0$ 
    -  Can solve by forcing labeled nodes to have initial class values instead of losing its own values
    
And the algorithm runs until we reach a certain number of iterations or hit a solution tolerance error. Here we may see error with fixing the value of Y0 for original, especially if there is a labelling error, that may propagate itself. So we change the algorithm to normalised Laplacian $L = D$<sup>$-1/2$</sup>$AD$<sup>$-1/2$</sup> and change our propagation agorithm to $Y^t = \alpha L Y$<sup>t-1</sup>$ + (1-\alpha)Y^0$ and stops when a certain conition is met. Here we have regulariser $\alpha \in [0,1]$ to weight influence of original solution at each iteration, imposing the "quality" of the original solution and its influence in the final solution.

## Graph regularisation methods
A loss function that depends on labeled and unlabelled samples, with the second (on unlablled) term acting as a regularising term that depends on the topological information of graph *G*. This can be powerful as a tool to regularise the training of neural networks. 

**inductive model**: Can be used on unobserved samples and does not require test samples to belong to input graph.

**Manifold learning**: A shallow form of learning whereby the the parameterised function does not leverage on any form of intermediate embeddings

-  Regularisation can be applied to final output of network
-  Regularisation applied to inetermediate layers, regularising the embedding representation
-  Regularisation applied to an auxiliary network that shares first k-1 layers; corresponds to training an unsupervised embedding network while simultaneously training a supervised network.

## Neural Graph Learning
Generalises previous formulations to make it possible to apply graph regularisation to any form of a NN. Can apply to any graph, natural or synthetic. We can also generate synthetic graphs with adversarial examples where samples are perturbed to maximise errors, allowing us to obtain models more robust against adversarially generated examples.

## Planetoid
Extend graph regularisation in order to account for higher-order proximities, we have Planetoid (Predicting labels and neighbours with emebddings trasductively or inductively from data), extends skip-gram for compute node embeddings to incorporate node-label information. Skip-gram methods are based on generating random walks thoruhg a graph then using the generated sequences to learn embeddings via a skip-gram model, we modify for supervised loss:
-  Softmax layer to predict graph context of sampled random-walk sequences
-  Set of hidden layers that combine together with hidden layers derived from node features to predict class labels

The cost function to be minimised is composed of supervised and unsupervised loss $L_s$, $L_u$ respectively. The unsupervised loss is analgous to the one used with skip-gram with negative sampling where the supervised loss minimises the conditional probability. However this formulation is transductive as it requires samples belonging to the graph to be applied; in semi-supervised task this can efficiently be used to predict labels for unlabeled examples. However, cannot be used for unobserved samples. There is an inductive version of Planetoid by parameterising the embeddings as a function of the node features, via dedicated connected layers.